In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████████████████████████| 30/30 [00:08<00:00,  3.51it/s]


eval images shape: (30, 1041, 1302) contain 10099 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.003125,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.25,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 115.0,
 'e_per_adu': 0.4,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/RN_e_venus.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6,
 'augment': True}

In [10]:
start_time = time.time()
model.fit(batch_size=32, max_iters=20000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 11648.861 || Time Upd.: 393.1 ms  || BatchNr.: 100
 || Cost: 408.374 || Time Upd.: 376.4 ms  || BatchNr.: 200
 || Cost: 105.216 || Time Upd.: 372.7 ms  || BatchNr.: 300
 || Cost: 64.121 || Time Upd.: 379.9 ms  || BatchNr.: 400
 || Cost: 51.340 || Time Upd.: 379.0 ms  || BatchNr.: 500
 || Cost: 42.780 || Time Upd.: 382.5 ms  || BatchNr.: 600
 || Cost: 45.144 || Time Upd.: 376.3 ms  || BatchNr.: 700
 || Cost: 40.832 || Time Upd.: 379.1 ms  || BatchNr.: 800
 || Cost: 36.594 || Time Upd.: 374.9 ms  || BatchNr.: 900
 || Cost: 44.136 || Time Upd.: 377.5 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:10123
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 10090
JoR: 1.093 || Eff_3d: 42.527 || Jaccard: 0.850 || Factor: 402.042 || RMSE_lat: 77.760 || RMSE_ax: 64.860 |

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9286
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9286
JoR: 1.195 || Eff_3d: 42.833 || Jaccard: 0.893 || Factor: 316.657 || RMSE_lat: 74.761 || RMSE_ax: 74.621 || RMSE_x: 53.666 || RMSE_y: 52.050 || Cost: 27.157 || Recall: 0.906 || Precision: 0.985 || BatchNr.: 2200 || Time Upd.: 372.6 ms 
 || Count Cost: 16.4712 || Localization Cost: 14.5957 || Bg Cost: 0.0485 || P_locs Cost: 21.5401saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 42.833 || 74.761 || 74.621 || 2200


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:11172
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 11153


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9720
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9720
JoR: 1.320 || Eff_3d: 50.855 || Jaccard: 0.921 || Factor: 327.006 || RMSE_lat: 69.793 || RMSE_ax: 53.862 || RMSE_x: 53.565 || RMSE_y: 44.743 || Cost: 22.766 || Recall: 0.941 || Precision: 0.978 || BatchNr.: 3400 || Time Upd.: 388.2 ms 
 || Count Cost: 14.3357 || Localization Cost: 11.6077 || Bg Cost: 0.0443 || P_locs Cost: 18.9986saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 50.855 || 69.793 || 53.862 || 3400


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9640
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9640
Jo

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9679
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9679
JoR: 1.376 || Eff_3d: 52.960 || Jaccard: 0.921 || Factor: 324.245 || RMSE_lat: 66.921 || RMSE_ax: 50.959 || RMSE_x: 50.814 || RMSE_y: 43.548 || Cost: 17.458 || Recall: 0.939 || Precision: 0.979 || BatchNr.: 4600 || Time Upd.: 380.7 ms 
 || Count Cost: 14.4072 || Localization Cost: 7.6521 || Bg Cost: 0.0283 || P_locs Cost: 18.6320saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 52.960 || 66.921 || 50.959 || 4600


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9938
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9938
JoR

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9596
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9596
JoR: 1.383 || Eff_3d: 53.086 || Jaccard: 0.919 || Factor: 320.640 || RMSE_lat: 66.486 || RMSE_ax: 51.236 || RMSE_x: 49.768 || RMSE_y: 44.085 || Cost: 16.217 || Recall: 0.934 || Precision: 0.983 || BatchNr.: 5800 || Time Upd.: 371.9 ms 
 || Count Cost: 13.9386 || Localization Cost: 6.5441 || Bg Cost: 0.0302 || P_locs Cost: 17.8800saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 53.086 || 66.486 || 51.236 || 5800


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9927
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9927
JoR

...

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9855
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9855
JoR: 1.437 || Eff_3d: 55.477 || Jaccard: 0.935 || Factor: 327.219 || RMSE_lat: 65.063 || RMSE_ax: 45.486 || RMSE_x: 49.377 || RMSE_y: 42.368 || Cost: 13.030 || Recall: 0.955 || Precision: 0.978 || BatchNr.: 14200 || Time Upd.: 383.0 ms 
 || Count Cost: 13.8191 || Localization Cost: 4.0079 || Bg Cost: 0.0237 || P_locs Cost: 17.4466saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 55.477 || 65.063 || 45.486 || 14200


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9947
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9947
J

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9870
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9870
JoR: 1.430 || Eff_3d: 54.742 || Jaccard: 0.934 || Factor: 319.437 || RMSE_lat: 65.345 || RMSE_ax: 47.921 || RMSE_x: 49.529 || RMSE_y: 42.625 || Cost: 12.639 || Recall: 0.955 || Precision: 0.977 || BatchNr.: 15400 || Time Upd.: 374.8 ms 
 || Count Cost: 13.7683 || Localization Cost: 3.6248 || Bg Cost: 0.0243 || P_locs Cost: 17.2748saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 54.742 || 65.345 || 47.921 || 15400


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9914
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9914
J

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9891
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9891
JoR: 1.420 || Eff_3d: 55.456 || Jaccard: 0.933 || Factor: 322.507 || RMSE_lat: 65.737 || RMSE_ax: 44.049 || RMSE_x: 49.911 || RMSE_y: 42.782 || Cost: 11.292 || Recall: 0.956 || Precision: 0.976 || BatchNr.: 16600 || Time Upd.: 377.0 ms 
 || Count Cost: 13.6639 || Localization Cost: 2.6960 || Bg Cost: 0.0201 || P_locs Cost: 17.0115saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 55.456 || 65.737 || 44.049 || 16600


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9826
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9826
J

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9776
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9776
JoR: 1.414 || Eff_3d: 53.227 || Jaccard: 0.935 || Factor: 328.558 || RMSE_lat: 66.122 || RMSE_ax: 52.645 || RMSE_x: 50.597 || RMSE_y: 42.569 || Cost: 11.058 || Recall: 0.951 || Precision: 0.983 || BatchNr.: 17800 || Time Upd.: 513.4 ms 
 || Count Cost: 13.5226 || Localization Cost: 2.5312 || Bg Cost: 0.0200 || P_locs Cost: 16.8953saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 53.227 || 66.122 || 52.645 || 17800


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:10052
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 10052

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9747
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9747
JoR: 1.444 || Eff_3d: 55.124 || Jaccard: 0.935 || Factor: 320.651 || RMSE_lat: 64.765 || RMSE_ax: 47.583 || RMSE_x: 49.313 || RMSE_y: 41.985 || Cost: 10.962 || Recall: 0.950 || Precision: 0.984 || BatchNr.: 19000 || Time Upd.: 503.7 ms 
 || Count Cost: 13.6092 || Localization Cost: 2.4079 || Bg Cost: 0.0203 || P_locs Cost: 16.8458saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 55.124 || 64.765 || 47.583 || 19000


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10099, preds:9778
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10099 ,preds: 9778
J